In [1]:
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.patches as patches

import colormaps as cmaps 
import numpy as np
import os
import math
from matplotlib.animation import FuncAnimation
from matplotlib.patches import Rectangle, FancyArrowPatch
import seaborn as sns


In [2]:
#input some set up
#target point(-2.85,0.108).position point(-2.85,-0.623),width0.71,height1.256,and the rader postion0.45,
# Define the target point
target_x = -2
rader_x = 0.45
target_y = 0

 # Define the vertices of the hexagon in (x,y) coordinate pairs
#vertices = [(-2.0035, -1.27), (-2.0035, 1.486), (-0.8405,  1.486), (-0.8405, 1.123), (0.8465, 1.123), (0.8465, -1.270)]
vertices = [(-2.5, -1.37), (-2.5, 1.486), (-0.04,  1.486), (-0.04, 1.123), (0.8465, 1.123), (0.8465, -1.37)]
# Define the target area position, width, and height.target point(-2.85,0.108).position point(-2.85,-0.623),width0.71,height1.256,and the rader postion error 0.32
rect_pos = (-2.2,-0.314 )
rect_width = 0.35
rect_height = 0.628

criterion_dis = 4
criterion_Yr = 0.314

x_range = (-2.8,1.8)
y_range = (-1.8,1.8)
time_range = (0,180)
Y_range = (0,0.5)

# Define the path to the directory containing the data, the path to store the output 
Baselinedata_dir = "D:/FOV2/Baselinedata/"
Baselinestore_dir = "D:/FOV2/BRWanalysis/data/Baseline/"

Rotationdata_dir = "D:/FOV2/Rotationdata/"
Rotationstore_dir = "D:/FOV2/BRWanalysis/data/Rotation/"

Washoutdata_dir = "D:/FOV2/Washoutdata/"
Washoutstore_dir = "D:/FOV2/BRWanalysis/data/Washout/"


ReRotationdata_dir = "D:/FOV2/ReRotationdata/"
ReRotationstore_dir = "D:/FOV2/BRWanalysis/data/ReRotaion/"

ReWashoutdata_dir = "D:/FOV2/ReWashoutdata/"
ReWashoutstore_dir = "D:/FOV2/BRWanalysis/data/ReWashout/"


In [3]:
ReWashout_new_df_dir = os.path.join("D:/FOV2/BRWanalysis/data/ReWashout/new_df/")
 # Create the directory if it does not exist
if not os.path.exists(ReWashout_new_df_dir):
    os.makedirs(ReWashout_new_df_dir)

All valid data points for each fish in the Rewashout session

In [4]:
# Loop over the fish IDs in Washout session
for fish_id in ['fish9','fish10']:
    # Create an empty list to store the data frames
    dfs = []

    # Loop over the directories in the data directory
    for dir_name in os.listdir(ReWashoutdata_dir):
        # Set the path to the directory
        dir_path = os.path.join(ReWashoutdata_dir, dir_name, fish_id)
        # Check if the directory exists
        if os.path.isdir(dir_path):
            # Loop over the CSV files in the directory
            for file_name in os.listdir(dir_path):
                if file_name.endswith(".csv"):
                    # Set the path to the CSV file
                    file_path = os.path.join(dir_path, file_name)
                    # Load the CSV file into a DataFrame
                    df = pd.read_csv(file_path)

                    # Add the date and file name columns
                    df["date"] = dir_name
                    df["experiment_id"] = file_name.split(".")[0]
                    df["fish_id"] = fish_id
                    df['trial_id'] = file_name.split('__')[-1].split('.')[0]
                    # Append the DataFrame to the list
                    dfs.append(df)

    # Concatenate the DataFrames in the list
    combined_df = pd.concat(dfs)

    # Remove junk frames with no goldfish activity detected
    new_df = combined_df[(combined_df["Xf"] != 0) & (combined_df["Yf"] != 0)]

    # Set the path to the CSV file to store the new_df
    new_df_path = os.path.join(ReWashout_new_df_dir, f"{fish_id}_ReWashout_new_df.csv")
    # Save the new_df to the CSV file
    new_df.to_csv(new_df_path, index=False)

In [5]:
# Create an empty list to store the DataFrames
ReWashoutdfs = []

# Loop over the fish IDs
for fish_id in ['fish9','fish10']:
    # Set the path to the new_df table for the fish
    new_df_path = os.path.join (ReWashout_new_df_dir, f"{fish_id}_ReWashout_new_df.csv")
    # Load the new_df table into a DataFrame
    df = pd.read_csv(new_df_path)

    # Append the DataFrame to the list
    ReWashoutdfs.append(df)

# Concatenate the DataFrames in the list
all_ReWashoutfish_df = pd.concat(ReWashoutdfs)

# Reset the DataFrame index to start from 0
all_ReWashoutfish_df = all_ReWashoutfish_df.reset_index(drop=True)
# Define the path to the CSV file to store the all_fish_df
all_ReWashoutfish_df_path = os.path.join(ReWashout_new_df_dir, "allReWashoutfish_df.csv")
# Save the all_fish_df to the CSV file
all_ReWashoutfish_df.to_csv(all_ReWashoutfish_df_path, index=False)

In [6]:
# create time column, calculate the real time of each experiment
all_ReWashoutfish_df['time'] = 0  # initialize time column with 0
all_ReWashoutfish_df['time'][0] = 0  # set first time value to 0

for i in range(1, len(all_ReWashoutfish_df)):
    diff = all_ReWashoutfish_df['t'][i] - all_ReWashoutfish_df['t'][i-1]
    if diff > 1.5 or diff < 0 :
        if all_ReWashoutfish_df['experiment_id'][i]!=all_ReWashoutfish_df['experiment_id'][i-1]:
            all_ReWashoutfish_df['time'][i] = 0 
        else:
            all_ReWashoutfish_df['time'][i] = all_ReWashoutfish_df['time'][i-1] + diff
    else:
        all_ReWashoutfish_df['time'][i] = all_ReWashoutfish_df['time'][i-1] + diff

# set last time value equal to second to last
all_ReWashoutfish_df['time'].iloc[-1] = all_ReWashoutfish_df['time'].iloc[-2]

C:\Users\86153\AppData\Local\Temp\ipykernel_47400\1061915221.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_ReWashoutfish_df['time'][0] = 0  # set first time value to 0
C:\Users\86153\AppData\Local\Temp\ipykernel_47400\1061915221.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_ReWashoutfish_df['time'][i] = all_ReWashoutfish_df['time'][i-1] + diff
C:\Users\86153\AppData\Local\Temp\ipykernel_47400\1061915221.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.09562110899999965' has dtype incompatible with int64, please explicitly

In [7]:
# create  column, calculate the real distance of robot of each experiment
all_ReWashoutfish_df['distance'] = 0  # initialize distance column with 0
all_ReWashoutfish_df['distance'][0] = 0
all_ReWashoutfish_df['AbsYr'] = 0  # initialize distance column with 0
all_ReWashoutfish_df['AbsYr'][0] = 0
all_ReWashoutfish_df['Yds'] = 0  # initialize distance column with 0
all_ReWashoutfish_df['Yds'][0] = 0
all_ReWashoutfish_df['FishV'] = 0  # initialize distance column with 0
all_ReWashoutfish_df['FishV'][0] = 0
all_ReWashoutfish_df['FishVds'] = 0  # initialize distance column with 0
all_ReWashoutfish_df['FishVds'][0] = 0
all_ReWashoutfish_df['RobotV'] = 0  # initialize distance column with 0
all_ReWashoutfish_df['RobotV'][0] = 0
all_ReWashoutfish_df['RobotVds'] = 0  # initialize distance column with 0
all_ReWashoutfish_df['RobotVds'][0] = 0

for i in range(1, len(all_ReWashoutfish_df)):
    distancestep = math.sqrt((all_ReWashoutfish_df['Xr'][i] - all_ReWashoutfish_df['Xr'][i-1])**2 + (all_ReWashoutfish_df['Yr'][i] - all_ReWashoutfish_df['Yr'][i-1])**2)
    AbsYrstep = np.abs(all_ReWashoutfish_df['Yr'][i])
    Ydsstep = (all_ReWashoutfish_df['Yr'][i])*distancestep
    FishVstep = all_ReWashoutfish_df['YAWf'][i]* 180 / np.pi + 180
    FishVstepds = (all_ReWashoutfish_df['YAWf'][i]* 180 / np.pi + 180)*distancestep
    RobotVstep = math.atan2(( all_ReWashoutfish_df['Yr'][i] - all_ReWashoutfish_df['Yr'][i-1]),(all_ReWashoutfish_df['Xr'][i] - all_ReWashoutfish_df['Xr'][i-1]))
    RobotVstepds = RobotVstep*distancestep

    if all_ReWashoutfish_df['time'][i] == 0:
        all_ReWashoutfish_df['distance'][i] = 0
        all_ReWashoutfish_df['AbsYr'][i]=0
        all_ReWashoutfish_df['Yds'][i]=0
        all_ReWashoutfish_df['FishV'][i] = 0
        all_ReWashoutfish_df['FishVds'][i] = 0
        all_ReWashoutfish_df['RobotV'][i] = 0
        all_ReWashoutfish_df['RobotVds'][i] = 0       
        

        
    else:
        all_ReWashoutfish_df['distance'][i] = all_ReWashoutfish_df['distance'][i-1] + distancestep
        all_ReWashoutfish_df['AbsYr'][i] = all_ReWashoutfish_df['AbsYr'][i-1] + AbsYrstep
        all_ReWashoutfish_df['Yds'][i] = all_ReWashoutfish_df['Yds'][i-1] + Ydsstep
        all_ReWashoutfish_df['FishV'][i] =  FishVstep
        all_ReWashoutfish_df['FishVds'][i] =  FishVstepds
        all_ReWashoutfish_df['RobotV'][i] = all_ReWashoutfish_df['RobotV'][i-1] + RobotVstep
        all_ReWashoutfish_df['RobotVds'][i] = all_ReWashoutfish_df['RobotVds'][i-1] + RobotVstepds      


# set last time value equal to second to last
all_ReWashoutfish_df['time'].iloc[-1] = all_ReWashoutfish_df['time'].iloc[-2]


C:\Users\86153\AppData\Local\Temp\ipykernel_47400\3160922666.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_ReWashoutfish_df['distance'][0] = 0
C:\Users\86153\AppData\Local\Temp\ipykernel_47400\3160922666.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_ReWashoutfish_df['AbsYr'][0] = 0
C:\Users\86153\AppData\Local\Temp\ipykernel_47400\3160922666.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_ReWashoutfish_df['Y

In [8]:
all_ReWashoutfish_df = all_ReWashoutfish_df.sort_values(by=['fish_id', 'date', 'trial_id'])

# Reset the index of the grouped DataFrame
all_ReWashoutfish_df.reset_index(inplace=True)

# Define the path to the CSV file to store the all_fish_df
all_ReWashoutfish_td_path = os.path.join(ReWashout_new_df_dir, "all_ReWashouttd_df.csv")
# Save the all_fish_df to the CSV file
all_ReWashoutfish_df.to_csv(all_ReWashoutfish_td_path, index=False)

In [9]:
# Convert 'date' column to datetime format
all_ReWashoutfish_df['date'] = pd.to_datetime(all_ReWashoutfish_df['date'])

# Create a boolean mask to identify the rows where df['time'] = 0 for the next row
mask = (all_ReWashoutfish_df['time'].shift(-1) == 0)



# Filter the DataFrame to keep only the last rows of each trial
filtered_all_ReWashoutfish_df = all_ReWashoutfish_df[mask]

# Initialize lists to store the calculated 'distance' and absolute maximum 'Yr' values
distance_values = []
abs_max_yr_values = []
trial_id_values= []
#FishAngle_values= []


# Loop through the filtered DataFrame and compute the 'distance' and absolute maximum 'Yr' values
for index, row in filtered_all_ReWashoutfish_df.iterrows():
    experiment_id = row['experiment_id']
    trial_df = filtered_all_ReWashoutfish_df [filtered_all_ReWashoutfish_df ['experiment_id'] == experiment_id]
    distance_values.append(trial_df.iloc[-1]['distance'])
    trial_id_values.append(trial_df['trial_id'])
    abs_max_yr_values.append(trial_df['Yr'].abs().max())
  #  FishAngle_values.append(trial_df[''])
# Create a new DataFrame with the required columns
ReWashout_trial_new_df = pd.DataFrame({
    'fish_id': filtered_all_ReWashoutfish_df['fish_id'],
    'date': filtered_all_ReWashoutfish_df['date'],
    'time': filtered_all_ReWashoutfish_df['time'],
    'Xr':filtered_all_ReWashoutfish_df['Xr'],
    'Yr':filtered_all_ReWashoutfish_df['Yr'],
    'experiment_id': filtered_all_ReWashoutfish_df['experiment_id'],
    'trialinterval':filtered_all_ReWashoutfish_df['trial_id'],
    'distance': distance_values,
    'AbsMax_Yr': abs_max_yr_values,
    'AbsYr_sum':filtered_all_ReWashoutfish_df['AbsYr'],
    'Yds_sum':filtered_all_ReWashoutfish_df['Yds'],
    'FishV_sum':filtered_all_ReWashoutfish_df['FishV'],
    'FishVds_sum':filtered_all_ReWashoutfish_df['FishVds'],
    'RobotV_sum':filtered_all_ReWashoutfish_df['RobotV'],
    'RobotVds_sum':filtered_all_ReWashoutfish_df['RobotVds'],
})

# Sort the DataFrame by 'fish_id' and 'date'
ReWashout_trial_new_df.sort_values(by=['fish_id', 'date'], inplace=True)

# Calculate the 'Day' column based on the date difference
ReWashout_trial_new_df['Day'] = (ReWashout_trial_new_df['date'] - ReWashout_trial_new_df.groupby('fish_id')['date'].transform('first')).dt.days + 1

# Add a new column 'Day' to represent the date as a number from 1 to the end for each fish
ReWashout_trial_new_df['Trialday'] = ReWashout_trial_new_df.groupby('fish_id')['date'].rank(method='dense').astype(int)


def calculate_success(row):
    if row['Xr'] < rect_pos[0]+rect_width and criterion_Yr*(-1)< row['Yr'] < criterion_Yr:
        return 1
    else:
        return 0
def calculate_excellent(row):
    if row['success'] == 1 and row['distance'] < criterion_dis:
        return 1
    else:
        return 0
# Apply the function to create the 'success' column
ReWashout_trial_new_df['success'] = ReWashout_trial_new_df.apply(calculate_success, axis=1)
ReWashout_trial_new_df['excellent'] = ReWashout_trial_new_df.apply(calculate_excellent, axis=1)


ReWashout_trial_path = r'D:\FOV2\BRWanalysis\data\ReWashout\all_ReWashouteachtrial.csv'
ReWashout_trial_new_df = ReWashout_trial_new_df.reset_index(drop=True)
ReWashout_trial_new_df.to_csv(ReWashout_trial_path, index=True)


In [10]:
ReWashout_trial_new_df['date'] = ReWashout_trial_new_df['date'].fillna('')
ReWashout_trial_new_df['date'] = ReWashout_trial_new_df['date'].astype(str)

In [11]:
# Initialize variables
ReWashout_sucedisday = 0
ReWashout_sucmaxAbsyday = 0
ReWashout_sucAbsYr_sumday = 0
ReWashout_sucYds_sumday = 0
ReWashout_sucFV_sumday = 0
ReWashout_sucFVds_sumday = 0
prev_date = None

# Lists to store cumulative values
ReWashout_cumulative_distances = []
ReWashout_cumulative_maxAbsy = []
ReWashout_cumulative_AbsYr_sum = []
ReWashout_cumulative_Yds_sum = []
ReWashout_cumulative_sucFV_sum = []
ReWashout_cumulative_sucFVds_sum = []

# Iterate through the DataFrame
for i in range(len(ReWashout_trial_new_df)):
    if ReWashout_trial_new_df['date'][i] != prev_date:
        # Date has changed, reset daily cumulative variables
        ReWashout_sucedisday = 0
        ReWashout_sucmaxAbsyday = 0
        ReWashout_sucAbsYr_sumday = 0
        ReWashout_sucYds_sumday = 0
        ReWashout_sucFV_sumday = 0
        ReWashout_sucFVds_sumday = 0

    if ReWashout_trial_new_df['success'][i] == 1:
        ReWashout_sucedisday_step = ReWashout_trial_new_df['distance'][i]
        ReWashout_sucmaxAbsyday_step = ReWashout_trial_new_df['AbsMax_Yr'][i]
        ReWashout_sucAbsYr_sumday_step = ReWashout_trial_new_df['AbsYr_sum'][i]
        ReWashout_sucYds_sumday_step = ReWashout_trial_new_df['Yds_sum'][i]
        ReWashout_sucFV_sumday_step = ReWashout_trial_new_df['FishV_sum'][i]
        ReWashout_sucFVds_sumday_step = ReWashout_trial_new_df['FishVds_sum'][i]
        ReWashout_sucedisday += ReWashout_sucedisday_step
        ReWashout_sucmaxAbsyday += ReWashout_sucmaxAbsyday_step
        ReWashout_sucAbsYr_sumday += ReWashout_sucAbsYr_sumday_step
        ReWashout_sucYds_sumday += ReWashout_sucYds_sumday_step
        ReWashout_sucFV_sumday += ReWashout_sucFV_sumday_step
        ReWashout_sucFVds_sumday += ReWashout_sucFVds_sumday_step
    else:
        ReWashout_sucedisday_step = 0
        ReWashout_sucmaxAbsyday_step = 0
        ReWashout_sucAbsYr_sumday_step = 0
        ReWashout_sucYds_sumday_step = 0
        ReWashout_sucFV_sumday_step = 0
        ReWashout_sucFVds_sumday_step = 0
        ReWashout_sucmaxAbsyday += ReWashout_sucmaxAbsyday_step
        ReWashout_sucAbsYr_sumday += ReWashout_sucAbsYr_sumday_step
        ReWashout_sucYds_sumday += ReWashout_sucYds_sumday_step
        ReWashout_sucFV_sumday += ReWashout_sucFV_sumday_step
        ReWashout_sucFVds_sumday += ReWashout_sucFVds_sumday_step

    # Append cumulative values
    ReWashout_cumulative_distances.append(ReWashout_sucedisday)
    ReWashout_cumulative_maxAbsy.append(ReWashout_sucmaxAbsyday)
    ReWashout_cumulative_AbsYr_sum.append(ReWashout_sucAbsYr_sumday)
    ReWashout_cumulative_Yds_sum.append(ReWashout_sucYds_sumday)
    ReWashout_cumulative_sucFV_sum.append(ReWashout_sucFV_sumday)
    ReWashout_cumulative_sucFVds_sum.append(ReWashout_sucFVds_sumday)

    prev_date = ReWashout_trial_new_df['date'][i]

# Add the cumulative columns to the DataFrame
ReWashout_trial_new_df['cumulative_success_distance'] = ReWashout_cumulative_distances
ReWashout_trial_new_df['cumulative_success_maxAbsy'] = ReWashout_cumulative_maxAbsy
ReWashout_trial_new_df['cumulative_success_AbsYr_sum'] = ReWashout_cumulative_AbsYr_sum
ReWashout_trial_new_df['cumulative_success_Yds_sum'] = ReWashout_cumulative_Yds_sum
ReWashout_trial_new_df['cumulative_success_sucFV_sum'] = ReWashout_cumulative_sucFV_sum
ReWashout_trial_new_df['cumulative_success_sucFVds_sum'] = ReWashout_cumulative_sucFVds_sum

# Group the original table by ['fish_id', 'date'] and calculate the average values for ['time'], ['distance'], and ['AbsMax_Yr']
ReWashout_grouped_data = ReWashout_trial_new_df.groupby(['fish_id', 'date', 'Day', 'Trialday']).agg({'time': 'mean', 'distance': 'mean', 'AbsMax_Yr': 'mean', 'AbsYr_sum': 'mean', 'Yds_sum': 'mean', 'FishV_sum': 'mean', 'FishVds_sum': 'mean', 'success': 'mean', 'excellent': 'mean'})

# Reset the index of the grouped DataFrame
ReWashout_grouped_data.reset_index(inplace=True)

# Calculate the time of success and the time of excellent
Wsum_data = ReWashout_trial_new_df.groupby(['fish_id', 'date', 'Day', 'Trialday'])[['success', 'excellent']].sum().reset_index()

# Create a new DataFrame with the required columns
ReWashout_new_table = pd.DataFrame({
    'fish_id': ReWashout_grouped_data['fish_id'],
    'date': ReWashout_grouped_data['date'],
    'Day': ReWashout_grouped_data['Day'],
    'Trialday': ReWashout_grouped_data['Trialday'],
    'average time': ReWashout_grouped_data['time'],
    'average distance': ReWashout_grouped_data['distance'],
    'average maximum deviation': ReWashout_grouped_data['AbsMax_Yr'],
    'average AbsYr_sum': ReWashout_grouped_data['AbsYr_sum'],
    'average Yds_sum': ReWashout_grouped_data['Yds_sum'],
    'average FishV_sum': ReWashout_grouped_data['FishV_sum'],
    'average FishVds_sum': ReWashout_grouped_data['FishVds_sum'],
    'average success rate': ReWashout_grouped_data['success'],
    'average excellent rate': ReWashout_grouped_data['excellent'],
    'timeofsuc': Wsum_data['success'],
    'timeofexc': Wsum_data['excellent'],
})

ReWashout_trial_new_df.to_csv(ReWashout_trial_path, index=False)
ReWashout_session_path = r'D:\FOV2\BRWanalysis\data\ReWashout\all_ReWashoutsession.csv'
ReWashout_new_table.to_csv(ReWashout_session_path, index=False)



In [12]:
# Create a mask to identify the rows where 'date' changes
ReWdate_change_mask = ReWashout_trial_new_df['date'] != ReWashout_trial_new_df['date'].shift(-1)

# Use the mask to extract the rows before the date changes
ReWrows_before_date_change = ReWashout_trial_new_df[ReWdate_change_mask]

# Reset the index of rows_before_date_change and summary DataFrames
ReWrows_before_date_change.reset_index(drop=True, inplace=True)

for i in range(len(ReWrows_before_date_change)):
    if ReWashout_new_table.loc[i, 'timeofsuc'] != 0:
        ReWrows_before_date_change.loc[i, 'average_success_distance'] = ReWrows_before_date_change.loc[i, 'cumulative_success_distance'] / ReWashout_new_table.loc[i, 'timeofsuc']
        ReWrows_before_date_change.loc[i, 'average_success_maxAbsy'] = ReWrows_before_date_change.loc[i, 'cumulative_success_maxAbsy'] / ReWashout_new_table.loc[i, 'timeofsuc']
        ReWrows_before_date_change.loc[i, 'average_success_AbsYr_sum'] = ReWrows_before_date_change.loc[i, 'cumulative_success_AbsYr_sum'] / ReWashout_new_table.loc[i, 'timeofsuc']
        ReWrows_before_date_change.loc[i, 'average_success_Yds_sum'] = ReWrows_before_date_change.loc[i, 'cumulative_success_Yds_sum'] / ReWashout_new_table.loc[i, 'timeofsuc']
        ReWrows_before_date_change.loc[i, 'average_success_FV_sum'] = ReWrows_before_date_change.loc[i, 'cumulative_success_sucFV_sum'] / ReWashout_new_table.loc[i, 'timeofsuc']
        ReWrows_before_date_change.loc[i, 'average_success_FVds_sum'] = ReWrows_before_date_change.loc[i, 'cumulative_success_sucFVds_sum'] / ReWashout_new_table.loc[i, 'timeofsuc']
    else:
        ReWrows_before_date_change.loc[i, 'average_success_distance'] = 0
        ReWrows_before_date_change.loc[i, 'average_success_maxAbsy'] = 0
        ReWrows_before_date_change.loc[i, 'average_success_AbsYr_sum'] = 0
        ReWrows_before_date_change.loc[i, 'average_success_Yds_sum'] = 0
        ReWrows_before_date_change.loc[i, 'average_success_FV_sum'] = 0
        ReWrows_before_date_change.loc[i, 'average_success_FVds_sum'] = 0

ReWrows_before_date_change = ReWrows_before_date_change.reset_index()

# Create a new DataFrame with the required columns
ReWashout_new_table = pd.DataFrame({
    'fish_id': ReWashout_grouped_data['fish_id'],
    'date': ReWashout_grouped_data['date'],
    'Day': ReWashout_grouped_data['Day'],
    'Trialday': ReWashout_grouped_data['Trialday'],
    'average time': ReWashout_grouped_data['time'],
    'average distance': ReWashout_grouped_data['distance'],
    'average distance': ReWashout_grouped_data['distance'],
    'average maximum deviation': ReWashout_grouped_data['AbsMax_Yr'],
    'average AbsYr_sum': ReWashout_grouped_data['AbsYr_sum'],
    'average Yds_sum': ReWashout_grouped_data['Yds_sum'],
    'average FishV_sum': ReWashout_grouped_data['FishV_sum'],
    'average FishVds_sum': ReWashout_grouped_data['FishVds_sum'],
    'average success rate': ReWashout_grouped_data['success'],
    'average excellent rate': ReWashout_grouped_data['excellent'],
    'timeofsuc': Wsum_data['success'],
    'timeofexc': Wsum_data['excellent'],
    'average_success_distance': ReWrows_before_date_change['average_success_distance'],
    'average_success_maxAbsy': ReWrows_before_date_change['average_success_maxAbsy'],
    'average_success_AbsYr_sum': ReWrows_before_date_change['average_success_AbsYr_sum'],
    'average_success_Yds_sum': ReWrows_before_date_change['average_success_Yds_sum'],
    'average_success_FV_sum': ReWrows_before_date_change['average_success_FV_sum'],
    'average_success_FVds_sum': ReWrows_before_date_change['average_success_FVds_sum']
})

ReWashout_trial_new_df.to_csv(ReWashout_trial_path, index=True)

ReWashout_new_table.to_csv(ReWashout_session_path, index=False)


C:\Users\86153\AppData\Local\Temp\ipykernel_47400\2473162730.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ReWrows_before_date_change.loc[i, 'average_success_distance'] = ReWrows_before_date_change.loc[i, 'cumulative_success_distance'] / ReWashout_new_table.loc[i, 'timeofsuc']
C:\Users\86153\AppData\Local\Temp\ipykernel_47400\2473162730.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ReWrows_before_date_change.loc[i, 'average_success_maxAbsy'] = ReWrows_before_date_change.loc[i, 'cumulative_succ

In [13]:
ReWashoutdata_dir = "D:/FOV2/ReWashoutdata/"
ReWashoutstore_dir = "D:/FOV2/BRWanalysis/data/ReWashout/"
ReWashoutplot_dir = "D:/FOV2/BRWanalysis/plot/ReWashout/"

In [14]:
allfish_ReW_point_df_path = os.path.join(ReWashoutstore_dir,'new_df/all_ReWashouttd_df.csv')


allfish_ReW_session_df_path = os.path.join(ReWashoutstore_dir,'all_ReWashoutsession.csv')

allfish_ReW_trial_df_path = os.path.join(ReWashoutstore_dir,'all_ReWashouteachtrial.csv')

allfish_ReW_point_df = pd.read_csv(allfish_ReW_point_df_path)

allfish_ReW_trial_df = pd.read_csv(allfish_ReW_trial_df_path)

allfish_ReW_session_df = pd.read_csv(allfish_ReW_session_df_path)

unique_ReW_fish = allfish_ReW_session_df['fish_id'].unique()
unique_ReW_dates = allfish_ReW_session_df['date'].unique()

In [15]:

def create_ReWarrow_plot(fish_id, date, trials):
    # Create a new figure and axes objects for the subplot
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Initialize a list to store Line2D objects for the legend
    legend_lines = []
    legend_labels = []
    
    # Loop over each experiment for the current date and fish
    for exp_id, experiment in enumerate(trials):
        # Filter the DataFrame for the current experiment
        exp_df = allfish_ReW_trial_df[(allfish_ReW_trial_df['date'] == date) & (allfish_ReW_trial_df['fish_id'] == fish_id) & (allfish_ReW_trial_df['experiment_id'] == experiment)]
        
        # Get the X and Y values for the experiment
        X = exp_df['Xr'].values
        Y = exp_df['Yr'].values
        
        # Determine the line color based on exp_id
        line_color = plt.cm.winter(exp_id / len(trials))
        
        # Plot arrows and add labels for each trial
        for x, y in zip(X, Y):
            ax.arrow(0, 0, x, y, head_width=0.05, head_length=0.1, fc=line_color, ec=line_color)
            ax.text(x, y, f"{math.degrees(math.atan2(y, x)):.2f}", fontsize=8, ha='right') 

        # Add legend information
        line = plt.Line2D([0], [0], color=line_color, label=f'Experiment {experiment}')
        legend_lines.append(line)
        legend_labels.append(f'Trial {exp_id + 1}')
                # Define the vertices of the hexagon in (x,y) coordinate pairs

      
        # Define the target area position, width, and height
        rect_pos = (-2.2, -0.314)
        rect_width = 0.35
        rect_height = 0.628

        # Create a rectangle patch with the specified properties
        rect = patches.Rectangle(rect_pos, rect_width, rect_height, linewidth=2, edgecolor='r', facecolor='none', alpha=0.5)
        # Create a polygon patch with the specified vertices
        hexagon = patches.Polygon(vertices, linewidth=2, edgecolor='r', facecolor='none', alpha=0.5)
        # Add the polygon patch to the subplot
        ax.add_patch(hexagon)
        # Add the rectangle patch to the subplot
        ax.add_patch(rect)
        ax.set_aspect('equal')

    # Set aspect ratio of the plot to be equal
    ax.set_aspect('equal')

    # Add a title to the subplot
    ax.set_title(f'{fish_id} - Date {date} - Arrow Plot')

    # Add legend to the subplot
    ax.legend(legend_lines, legend_labels, loc='center left', bbox_to_anchor=(1, 0.5))
    # Set fixed data ranges for both axes
    plt.xlim(x_range)
    plt.ylim(y_range)
    # Save the plot with a custom filename
    plt.savefig(f'D:\\FOV2\\BRWanalysis\\plots\\ReWashout\\{fish_id}\\{date}_{fish_id}_arrow_plot.png')
    
    # Close the current figure to avoid overlapping plots
    plt.close()


In [16]:
def create_ReWpoint_plot(fish_id, date, trials):
    # Create a new figure and axes objects for the subplot
    fig, ax = plt.subplots(figsize=(2.3, 2.3))
    
    # Initialize a list to store Line2D objects for the legend
    legend_lines = []
    legend_labels = []
    
    # Loop over each experiment for the current date and fish
    for exp_id, experiment in enumerate(trials):
        # Filter the DataFrame for the current experiment
        exp_df = allfish_ReW_point_df[(allfish_ReW_point_df['date'] == date) & (allfish_ReW_point_df['fish_id'] == fish_id) & (allfish_ReW_point_df['experiment_id'] == experiment)]
        
        # Get the X and Y values for the experiment
        X = exp_df['Xr'].values
        Y = exp_df['Yr'].values
        # Determine the line color based on exp_id
        #line_color = plt.cm.jet(exp_id / len(experiments))  # You can use a different colormap if needed
        line_color = plt.cm.winter(exp_id / len(trials))

        
        # Plot the experiment on the subplot and store the Line2D object in the list
        line, = ax.plot(X, Y, label=f'Trial {exp_id + 1}', color=line_color)
        
        ax.scatter(X[0], Y[0], color='#F0E442', s=30)  # Start point is green
        ax.scatter(X[-1], Y[-1], color='#CC79A7', s=30)  # End point is red
        legend_lines.append(line)
        legend_labels.append(f'Trial {exp_id + 1}')

        # Define the vertices of the hexagon in (x,y) coordinate pairs

      
        # Define the target area position, width, and height
        rect_pos = (-2.2, -0.314)
        rect_width = 0.35
        rect_height = 0.628

        # Create a rectangle patch with the specified properties
        rect = patches.Rectangle(rect_pos, rect_width, rect_height, linewidth=2, edgecolor='#CC79A7', facecolor='none', alpha=0.5)
        # Create a polygon patch with the specified vertices
        hexagon = patches.Polygon(vertices, linewidth=2, edgecolor='black', facecolor='none', alpha=0.5)
               # Add the polygon patch to the subplot
        ax.add_patch(hexagon)
        # Add the rectangle patch to the subplot
        ax.add_patch(rect)
        ax.set_aspect('equal')

    # # Add a title to the subplot
    # ax.set_title(f'{fish_id} - Date {date} - Washout')
    # Set fixed data ranges for both axes
    plt.xlim(x_range)
    plt.ylim(y_range)

    # # Add a legend to the subplot
    # ax.legend(legend_lines, legend_labels)
    # Add legend to the subplot
    # ax.legend(legend_lines, legend_labels, loc='center left', bbox_to_anchor=(1, 0.5))
    plt.grid(False)
    # Save the plot with a custom filename
    plt.savefig(f'D:\\FOV2\\BRWanalysis\\plots\\ReWashout\\{fish_id}\\{date}_{fish_id}_ReWashout.png',  bbox_inches='tight',dpi=600)
    
    # Close the current figure to avoid overlapping plots
    plt.close()

In [17]:
for fish_id in unique_ReW_fish:
    
    for date in unique_ReW_dates:
    #for date in ['2023-10-04']:
        # Filter the DataFrame for the current fish and date
        trials = allfish_ReW_point_df[(allfish_ReW_point_df['fish_id'] == fish_id) & (allfish_ReW_point_df['date'] == date)]['experiment_id'].unique()
        
        if len(trials) > 0:
            create_ReWpoint_plot(fish_id, date, trials)
            # create_ReWarrow_plot(fish_id, date, trials)